In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-10 11:31:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-03-10 11:31:25 (6.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [ ]:
# STEP 1
# Load in a sql function to use columns 
from pyspark.sql.functions import col

# Filter data and create a new DataFrame to retrieve all rows were total_votes is greater than or equal to 20 
vine_df = df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine','verified_purchase']).filter(col("total_votes") >= 20)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [ ]:
# STEP 2
# Filter the vine_df to create a new DataFrame that retrieves all rows where the number of helpful_votes divided by total_votes is greater than or equal to 50%
filtered_df = vine_df.filter(col("helpful_votes")/col("total_votes") >= 0.50)
filtered_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [ ]:
# STEP 3
# Filter the DataFrame for reviews that were written as part of the Vine program (paid)
vine_paid_df = filtered_df.filter(col("vine") == 'Y')
vine_paid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

In [ ]:
# STEP 4
# Filter the DataFrame for reviews that were written as not part of the Vine program (unpaid)
unpaid_df = filtered_df.filter(col("vine") == 'N')
unpaid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [ ]:
# STEP 5.1  
# Determine the total number of reviews (total_votes >= 20 & helpful_votes/total_votes >= 0.50)
filtered_df.count()

121360

In [ ]:
# STEP 5.2
# Determine the number of 5-star reviews 
filtered_df.filter(col("star_rating") == 5).count()

74690

In [ ]:
# STEP 5.3
# Determine the percentage of 5-star reviews for the two types of reviews (paid)
# Number of 5 star reviews that were Vine reviews
vine_five_star = vine_paid_df.filter(col("star_rating") == 5).count()
vine_five_star

220

In [ ]:
# Total number of Vine reviews
vine_paid_df.count()

497

In [ ]:
# Percentage of total reviews that were Vine reviews 
vine_review_percentage = vine_paid_df.count() / filtered_df.count()
vine_review_percentage

0.0040952537903757415

In [ ]:
# Percentage of Vine reviews that were 5 stars
vine_five_star_percentage = vine_five_star / vine_paid_df.count()
vine_five_star_percentage

0.4426559356136821

In [ ]:
# STEP 5.4
# Determine the percentage of 5-star reviews for the two types of reviews (unpaid)
# Number of 5 star reviews that were unpaid reviews 
unpaid_five_star = unpaid_df.filter(col("star_rating") == 5).count()
unpaid_five_star

74470

In [ ]:
# Total number of unpaid reviews
unpaid_df.count()

120863

In [ ]:
# Percentage of total reviews that were Vine reviews 
unpaid_review_percentage = unpaid_df.count() / filtered_df.count()
unpaid_review_percentage

0.9959047462096242

In [ ]:
# Percentage of unpaid reviews that were 5 stars
unpaid_five_star_percentage = unpaid_five_star / unpaid_df.count()
unpaid_five_star_percentage

0.6161521722942506